In [9]:
import spacy
import numpy as np
import pandas as pd
from nltk import word_tokenize
nlp = spacy.load('ru_core_news_sm')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [10]:
%pip install spacy[ru]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
data = pd.read_csv('banks.csv',sep ="\t")

In [9]:
class Tokenizer:
    def __init__(self):
        self.nlp = spacy.load('ru_core_news_sm')
    
    def tokenize_text(self, text):
        tokens = []
        for sentence in self.nlp(text).sents:
            for token in sentence:
                if not token.is_stop and not token.is_punct:
                    tokens.append(token.text)
        return tokens

X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Score'], test_size=0.5, random_state=35)

tokenizer = Tokenizer()
new_data = pd.DataFrame({'text': X_train, 'tokenizer_text': X_train.apply(tokenizer.tokenize_text)})
new_data

,text,tokenizer_text
8658,"Мне прискорбно заявлять,что некогда уважаемому...","[прискорбно, заявлять, уважаемому, Альфа, Банк..."
7641,3 октября 2016 года после полудня (точное врем...,"[3, октября, 2016, года, полудня, точное, врем..."
7269,Была заказана карта Бинбанк Visa Platinum по а...,"[заказана, карта, Бинбанк, Visa, Platinum, акц..."
2688,Ну что я могу сказать?.. ТКС работает на 5+! П...,"[сказать, ТКС, работает, 5, +, крайней, мере, ..."
4917,"Открыл рублевый дебетовый счет в Альфе,пользов...","[Открыл, рублевый, дебетовый, счет, Альфе, пол..."
...,...,...
3007,"Уважаемый Альфа Банк,вот уже четвертый месяц в...","[Уважаемый, Альфа, Банк, четвертый, месяц, Сэб..."
7148,Являюсь клиентом ПАО БИНБАНК. 24.04.2017 в 18 ...,"[Являюсь, клиентом, ПАО, БИНБАНК, 24.04.2017, ..."
9143,"Что же это такое творится,господа банкиры!?Два...","[творится, господа, банкиры!?Два, обращения, к..."
1295,Я устал и сил моральных общаться с банком нет ...,"[устал, сил, моральных, общаться, банком, уйти..."


In [17]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['Score'] = data['Score'].apply(lambda x: 1 if x == 'Positive' else 0)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize_text)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

logistic_model = make_pipeline(LogisticRegression())
logistic_model.fit(X_train_vectorized, y_train)

y_pred_logistic = logistic_model.predict(X_test_vectorized)
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print('Accuracy (Logistic Regression):', accuracy_logistic)

print('\nClassification Report (Logistic Regression):\n', classification_report(y_test, y_pred_logistic))

Accuracy (Logistic Regression): 0.9387055293613373

Classification Report (Logistic Regression):
               precision    recall  f1-score   support

    Negative       0.93      0.95      0.94      3501
    Positive       0.95      0.93      0.94      3498

    accuracy                           0.94      6999
   macro avg       0.94      0.94      0.94      6999
weighted avg       0.94      0.94      0.94      6999



In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# использование НС

In [12]:

X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Score'], test_size=0.5, random_state=35)

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)

In [15]:
X_train_padded = pad_sequences(X_train_tokenized)
X_test_padded = pad_sequences(X_test_tokenized, maxlen=X_train_padded.shape[1])

In [18]:
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index)+1, 100, input_length=X_train_padded.shape[1]),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit(X_train_padded, y_train_encoded, epochs=10, batch_size=32)

Epoch 1/10
219/219 [==============================] - 69s 314ms/step - loss: 0.3212 - accuracy: 0.8818
Epoch 2/10
219/219 [==============================] - 71s 323ms/step - loss: 0.1057 - accuracy: 0.9650
Epoch 3/10
219/219 [==============================] - 71s 325ms/step - loss: 0.0251 - accuracy: 0.9961
Epoch 4/10
219/219 [==============================] - 72s 329ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 5/10
219/219 [==============================] - 73s 332ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 6/10
219/219 [==============================] - 74s 337ms/step - loss: 9.9274e-04 - accuracy: 1.0000
Epoch 7/10
219/219 [==============================] - 74s 337ms/step - loss: 6.4537e-04 - accuracy: 1.0000
Epoch 8/10
219/219 [==============================] - 73s 335ms/step - loss: 4.5065e-04 - accuracy: 1.0000
Epoch 9/10
219/219 [==============================] - 73s 333ms/step - loss: 3.2948e-04 - accuracy: 1.0000
Epoch 10/10
219/219 [==============================] - 73

In [23]:
_, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print('Accuracy: %.1f' % (accuracy * 100))

219/219 [==============================] - 9s 42ms/step - loss: 0.2319 - accuracy: 0.9363
Accuracy: 93.6
